In [16]:

#dog breed indentifier using CNN
# Importing the libraries
import tensorflow as tf


# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# print(gpu_devices)

# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# This disables the GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except e:
    # Invalid device or cannot modify virtual devices once initialized.
    print(e)
    pass

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import cv2
import random
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

import time
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy


In [17]:
RESULTS_DIR = './results/'
IMG_SIZE = 224

BATCH_SIZE_PRE = 32
BATCH_SIZE_FINE = 32
EPOCHS_PRE = 10
EPOCHS_FINE = 12
MOMENTUM = 0.9

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range = 10,
    horizontal_flip = True,
    validation_split = 0.2)

train_pre_generator = train_datagen.flow_from_directory(
    directory = "./data/fusion/train/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_PRE,
    class_mode = "categorical")

train_fine_generator = train_datagen.flow_from_directory(
    directory = "./data/fusion/train/",
    target_size = (IMG_SIZE, IMG_SIZE),
    color_mode = "rgb",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical")

test_datagen = ImageDataGenerator(
    rescale=1.0/255
)
test_generator = test_datagen.flow_from_directory(
    directory = "./data/fusion/test/",
    batch_size = BATCH_SIZE_FINE,
    class_mode = "categorical"
)


Found 18680 images belonging to 172 classes.
Found 18680 images belonging to 172 classes.
Found 9355 images belonging to 172 classes.


In [19]:
assert len(os.listdir('./data/fusion/train/')) == len(os.listdir('./data/fusion/test/'))
num_breeds = len(os.listdir('./data/fusion/train/')) - 1
print("num_breeds:", num_breeds)

num_breeds: 172


In [21]:
nasnetmobile_base_model = NASNetMobile(include_top=False,
                                        weights='imagenet',
                                        input_shape=(IMG_SIZE,IMG_SIZE,3))

x_base_nasnetmobile = nasnetmobile_base_model.output
x_avg_pooling_nasnetmobile = GlobalAveragePooling2D()(x_base_nasnetmobile)
x_first_fc_nasnetmobile = Dense(1024, activation='relu')(x_avg_pooling_nasnetmobile)
x_last_fc_nasnetmobile = Dense(num_breeds, activation='softmax')(x_first_fc_nasnetmobile)

nasnetmobile_custom_model = Model(inputs=nasnetmobile_base_model.inputs, outputs=x_last_fc_nasnetmobile)

nasnetmobile_custom_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                           

                                                                                                  
 separable_conv_1_pad_reduction  (None, 115, 115, 32  0          ['activation_383[0][0]']         
 _right3_stem_1 (ZeroPadding2D)  )                                                                
                                                                                                  
 activation_385 (Activation)    (None, 56, 56, 11)   0           ['reduction_add_1_stem_1[0][0]'] 
                                                                                                  
 separable_conv_1_bn_reduction_  (None, 56, 56, 11)  44          ['separable_conv_1_reduction_righ
 right2_stem_1 (BatchNormalizat                                  t2_stem_1[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable

 add_8 (Add)                    (None, 56, 56, 11)   0           ['reduction_add_2_stem_1[0][0]', 
                                                                  'reduction_left4_stem_1[0][0]'] 
                                                                                                  
 reduction_add4_stem_1 (Add)    (None, 56, 56, 11)   0           ['separable_conv_2_bn_reduction_l
                                                                 eft4_stem_1[0][0]',              
                                                                  'reduction_right5_stem_1[0][0]']
                                                                                                  
 cropping2d_8 (Cropping2D)      (None, 111, 111, 32  0           ['zero_padding2d_8[0][0]']       
                                )                                                                 
                                                                                                  
 reduction

 separable_conv_2_bn_reduction_  (None, 28, 28, 22)  88          ['separable_conv_2_reduction_righ
 right1_stem_2 (BatchNormalizat                                  t1_stem_2[0][0]']                
 ion)                                                                                             
                                                                                                  
 separable_conv_1_pad_reduction  (None, 61, 61, 22)  0           ['activation_392[0][0]']         
 _right2_stem_2 (ZeroPadding2D)                                                                   
                                                                                                  
 activation_394 (Activation)    (None, 56, 56, 22)   0           ['adjust_bn_stem_2[0][0]']       
                                                                                                  
 reduction_add_1_stem_2 (Add)   (None, 28, 28, 22)   0           ['separable_conv_2_bn_reduction_l
          

                                                                                                  
 separable_conv_2_bn_reduction_  (None, 28, 28, 22)  88          ['separable_conv_2_reduction_left
 left4_stem_2 (BatchNormalizati                                  4_stem_2[0][0]']                 
 on)                                                                                              
                                                                                                  
 reduction_right5_stem_2 (MaxPo  (None, 28, 28, 22)  0           ['reduction_pad_1_stem_2[0][0]'] 
 oling2D)                                                                                         
                                                                                                  
 zero_padding2d_9 (ZeroPadding2  (None, 57, 57, 44)  0           ['adjust_relu_1_0[0][0]']        
 D)                                                                                               
          

...

 separable_conv_1_normal_left1_  (None, 7, 7, 176)   35376       ['activation_529[0][0]']         
 10 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_1_normal_right1  (None, 7, 7, 176)   32560       ['activation_531[0][0]']         
 _10 (SeparableConv2D)                                                                            
                                                                                                  
 separable_conv_1_normal_left2_  (None, 7, 7, 176)   35376       ['activation_533[0][0]']         
 10 (SeparableConv2D)                                                                             
                                                                                                  
 separable_conv_1_normal_right2  (None, 7, 7, 176)   32560       ['activation_535[0][0]']         
 _10 (Sepa

                                                                                                  
 normal_add_1_10 (Add)          (None, 7, 7, 176)    0           ['separable_conv_2_bn_normal_left
                                                                 1_10[0][0]',                     
                                                                  'separable_conv_2_bn_normal_righ
                                                                 t1_10[0][0]']                    
                                                                                                  
 normal_add_2_10 (Add)          (None, 7, 7, 176)    0           ['separable_conv_2_bn_normal_left
                                                                 2_10[0][0]',                     
                                                                  'separable_conv_2_bn_normal_righ
                                                                 t2_10[0][0]']                    
          

 activation_542 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_left
                                                                 1_11[0][0]']                     
                                                                                                  
 activation_544 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_righ
                                                                 t1_11[0][0]']                    
                                                                                                  
 activation_546 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_left
                                                                 2_11[0][0]']                     
                                                                                                  
 activation_548 (Activation)    (None, 7, 7, 176)    0           ['separable_conv_1_bn_normal_righ
          

 activation_552 (Activation)    (None, 7, 7, 1056)   0           ['normal_concat_11[0][0]']       
                                                                                                  
 adjust_conv_projection_12 (Con  (None, 7, 7, 176)   185856      ['activation_551[0][0]']         
 v2D)                                                                                             
                                                                                                  
 normal_conv_1_12 (Conv2D)      (None, 7, 7, 176)    185856      ['activation_552[0][0]']         
                                                                                                  
 adjust_bn_12 (BatchNormalizati  (None, 7, 7, 176)   704         ['adjust_conv_projection_12[0][0]
 on)                                                             ']                               
                                                                                                  
 normal_bn

 separable_conv_2_bn_normal_lef  (None, 7, 7, 176)   704         ['separable_conv_2_normal_left1_1
 t1_12 (BatchNormalization)                                      2[0][0]']                        
                                                                                                  
 separable_conv_2_bn_normal_rig  (None, 7, 7, 176)   704         ['separable_conv_2_normal_right1_
 ht1_12 (BatchNormalization)                                     12[0][0]']                       
                                                                                                  
 separable_conv_2_bn_normal_lef  (None, 7, 7, 176)   704         ['separable_conv_2_normal_left2_1
 t2_12 (BatchNormalization)                                      2[0][0]']                        
                                                                                                  
 separable_conv_2_bn_normal_rig  (None, 7, 7, 176)   704         ['separable_conv_2_normal_right2_
 ht2_12 (B

In [23]:
#Train model - freeze body layers first
for layer in nasnetmobile_base_model.layers:
    layer.trainable = False

nasnetmobile_custom_model.compile(optimizer=SGD(momentum=MOMENTUM), loss=categorical_crossentropy, metrics=['accuracy'])

model_train_pre = nasnetmobile_custom_model.fit_generator(
    generator = train_pre_generator,
    steps_per_epoch = (train_pre_generator.n//train_pre_generator.batch_size),
    epochs = EPOCHS_PRE,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose=1
)

nasnetmobile_custom_model.save(os.path.join(RESULTS_DIR, 'model_pre_nasnetm_3.h5'))

/tmp/ipykernel_359351/64620997.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_pre = nasnetmobile_custom_model.fit_generator(


Epoch 1/10
583/583 [==============================] - 343s 581ms/step - loss: 0.7810 - accuracy: 0.7591 - val_loss: 0.7332 - val_accuracy: 0.8092
Epoch 2/10
583/583 [==============================] - 337s 579ms/step - loss: 0.6755 - accuracy: 0.7855 - val_loss: 0.6775 - val_accuracy: 0.8105
Epoch 3/10
583/583 [==============================] - 337s 579ms/step - loss: 0.6179 - accuracy: 0.8030 - val_loss: 0.6866 - val_accuracy: 0.8153
Epoch 4/10
583/583 [==============================] - 335s 575ms/step - loss: 0.5692 - accuracy: 0.8177 - val_loss: 0.6790 - val_accuracy: 0.8090
Epoch 5/10
583/583 [==============================] - 335s 575ms/step - loss: 0.5288 - accuracy: 0.8270 - val_loss: 0.6868 - val_accuracy: 0.8059
Epoch 6/10
583/583 [==============================] - 333s 572ms/step - loss: 0.4840 - accuracy: 0.8414 - val_loss: 0.7028 - val_accuracy: 0.8019
Epoch 7/10
583/583 [==============================] - 339s 582ms/step - loss: 0.4515 - accuracy: 0.8478 - val_loss: 0.6840 -

In [14]:
#Train model - train all layers
for layer in nasnetmobile_base_model.layers:
    layer.trainable = True
    
nasnetmobile_custom_model.compile(optimizer=SGD(momentum=MOMENTUM), loss=categorical_crossentropy, metrics=['accuracy'] )

model_train_fine = nasnetmobile_custom_model.fit_generator(
    generator = train_fine_generator,
    steps_per_epoch = (train_fine_generator.n//train_fine_generator.batch_size),
    epochs = EPOCHS_FINE,
    validation_data = test_generator,
    validation_steps = (test_generator.n//test_generator.batch_size),
    verbose = 1
)

nasnetmobile_custom_model.save(os.path.join(RESULTS_DIR, 'model_fine_nasnetm_2.h5'))

/tmp/ipykernel_359351/738299973.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_train_fine = nasnetmobile_custom_model.fit_generator(


Epoch 1/12
583/583 [==============================] - 1065s 2s/step - loss: 1.4879 - accuracy: 0.5785 - val_loss: 4.0172 - val_accuracy: 0.1415
Epoch 2/12
583/583 [==============================] - 1045s 2s/step - loss: 1.0001 - accuracy: 0.6960 - val_loss: 3.2852 - val_accuracy: 0.2521
Epoch 3/12
583/583 [==============================] - 1045s 2s/step - loss: 0.8071 - accuracy: 0.7455 - val_loss: 3.4898 - val_accuracy: 0.2308
Epoch 4/12
583/583 [==============================] - 1046s 2s/step - loss: 0.6637 - accuracy: 0.7891 - val_loss: 3.3886 - val_accuracy: 0.2414
Epoch 5/12
583/583 [==============================] - 1046s 2s/step - loss: 0.5473 - accuracy: 0.8214 - val_loss: 2.8758 - val_accuracy: 0.3243
Epoch 6/12
583/583 [==============================] - 1045s 2s/step - loss: 0.4652 - accuracy: 0.8496 - val_loss: 2.6710 - val_accuracy: 0.3561
Epoch 7/12
583/583 [==============================] - 1042s 2s/step - loss: 0.3868 - accuracy: 0.8749 - val_loss: 2.9753 - val_accuracy:

In [24]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * Y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives/(possible_positives + K.epsilon())
    return recall

# From Ananya, f-1 score
nasnetmobile_custom_model= load_model(os.path.join(RESULTS_DIR, 'model_pre_mbnet2_2.h5'))
y_pred = nasnetmobile_custom_model.predict_generator(test_generator, steps=test_generator.samples // BATCH_SIZE_FINE, verbose=1)

y_pred = np.argmax(y_pred, axis=1)

y_true = test_generator.classes


print('f1 score:', f1_score(y_true, y_pred))

/tmp/ipykernel_359351/86194569.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = nasnetmobile_custom_model.predict_generator(test_generator, steps=test_generator.samples // BATCH_SIZE_FINE, verbose=1)


292/292 [==============================] - 89s 304ms/step


NameError: name 'K' is not defined